# Carga/Consumo

Primeiro modelo (não utilizado)

In [230]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Get today's date
start_date = '2019-01-01'
end_date = datetime.now()

df_carga = pd.DataFrame()

# Function to obtain data from the API
def obter_carga_verificada(api_url, data_inicio, data_fim, cod_areacarga):
    url = f'{api_url}?dat_inicio={data_inicio}&dat_fim={data_fim}&cod_areacarga={cod_areacarga}'
    
    try:
        # Making the GET request to the API
        resposta = requests.get(url)
        resposta.raise_for_status()  # Raise an error for bad responses

        # Getting the content of the response in JSON format
        dados = resposta.json()
        
        return dados
    except requests.exceptions.HTTPError as errh:
        print(f"Erro HTTP: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Erro de Conexão: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Tempo de Conexão Esgotado: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Erro ao fazer a requisição: {err}")
    
    return None  # Return None if an error occurs

# Defining the API URL and query parameters
api_url = 'https://apicarga.ons.org.br/prd/cargaverificada'
  # Replace 'AREA' with the desired area code

def generate_date_ranges(start_date_str, finish_date):
    # Parse the start date string into a datetime object
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')

    # Initialize the current start date
    current_start_date = start_date

    while True:
        # Calculate the finish date for the current range
        current_finish_date = current_start_date + timedelta(days=90)  # Approximately 3 months

        # Format the finish date as a string
        end = current_finish_date.strftime('%Y-%m-%d')

        # Stop if the current finish date exceeds the given finish date
        if current_finish_date > finish_date:
            break

        # Get data from the API for the current date range
        dados_carga = obter_carga_verificada(api_url, current_start_date.strftime('%Y-%m-%d'), end, cod_areacarga)
        
        # If the data is valid, concatenate it to the DataFrame
        if dados_carga is not None:
            df_temp = pd.DataFrame(dados_carga)  # Assuming the data is in a suitable format for DataFrame
            global df_carga  # Use the global DataFrame
            df_carga = pd.concat([df_carga, df_temp], ignore_index=True)

        # Update the current start date for the next iteration
        current_start_date = current_finish_date

# Call the function with the current date as the finish date


def values_treatment (df_carga):
    df_carga = df_carga[df_carga['val_cargaglobal'] != 0]
    df_carga = df_carga.drop_duplicates()
    df_carga['din_referenciautc'] = pd.to_datetime(df_carga['din_referenciautc'])

    # Extract year, month, day, and hour for grouping
    df_carga['year'] = df_carga['din_referenciautc'].dt.year
    df_carga['month'] = df_carga['din_referenciautc'].dt.month
    df_carga['day'] = df_carga['din_referenciautc'].dt.day
    df_carga['hour'] = df_carga['din_referenciautc'].dt.hour

    # Group by year, month, day, and hour, then sum the values and keep other columns
    result = df_carga.groupby(['year', 'month', 'day', 'hour'], as_index=False).agg(
        val_cargaglobal=('val_cargaglobal', 'mean'),
        val_cargaglobalcons=('val_cargaglobalcons', 'mean'),
        val_cargaglobalsmmgd=('val_cargaglobalsmmgd', 'mean'),
        val_cargasupervisionada=('val_cargasupervisionada', 'mean'),
        val_carganaosupervisionada=('val_carganaosupervisionada', 'mean'),
        val_cargammgd=('val_cargammgd', 'mean'),
        val_consistencia=('val_consistencia', 'mean'),
        cod_areacarga=('cod_areacarga', 'first'),  # Keep the first occurrence
        din_atualizacao=('din_atualizacao', 'first'),    # Keep the first occurrence
        dat_referencia=('dat_referencia', 'first')  # Keep the first occurrence
    )

    # Create a new datetime column
    result['din_referenciautc'] = pd.to_datetime(result[['year', 'month', 'day', 'hour']].assign(minute=0))

    # Convert back to the original string format
    result['din_referenciautc'] = result['din_referenciautc'].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')

    # Drop the year, month, day, hour columns if no longer needed
    result = result.drop(columns=['year', 'month', 'day', 'hour'])

    return result
cod_areacarga = 'SECO'
generate_date_ranges(start_date, end_date)
resultSECO = values_treatment(df_carga)

df_carga = pd.DataFrame()
cod_areacarga = 'NE'
generate_date_ranges(start_date, end_date)
resultNE = values_treatment(df_carga)

df_carga = pd.DataFrame()
cod_areacarga = 'N'
generate_date_ranges(start_date, end_date)
resultN = values_treatment(df_carga)

df_carga = pd.DataFrame()
cod_areacarga = 'S'
generate_date_ranges(start_date, end_date)
resultS = values_treatment(df_carga)
# Display the resulting DataFrame

df_carga = pd.concat([resultSECO, resultNE, resultN, resultS])


new_order = ['cod_areacarga', 'din_atualizacao', 'dat_referencia', 'din_referenciautc', 'val_cargaglobal', 'val_cargaglobalcons', 'val_cargaglobalsmmgd', 'val_cargasupervisionada', 'val_carganaosupervisionada', 'val_cargammgd', 'val_consistencia']
df_carga = df_carga[new_order]
df_carga = df_carga.drop_duplicates().reset_index(drop=True)
df_carga

Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 318)
Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 316)
Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 315)
Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 313)


,cod_areacarga,din_atualizacao,dat_referencia,din_referenciautc,val_cargaglobal,val_cargaglobalcons,val_cargaglobalsmmgd,val_cargasupervisionada,val_carganaosupervisionada,val_cargammgd,val_consistencia
0,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T03:00:00.000Z,35361.2270,35361.2270,35315.8360,35315.8360,0.00000,45.38990,0.0
1,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T04:00:00.000Z,34080.9905,34080.9905,34035.5995,34035.5995,0.00000,45.39010,0.0
2,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T05:00:00.000Z,32914.8085,32914.8085,32869.4195,32869.4195,0.00000,45.38975,0.0
3,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T06:00:00.000Z,32271.3705,32271.3705,32225.9795,32225.9795,0.00000,45.39100,0.0
4,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T07:00:00.000Z,32112.6400,32112.6400,32067.2525,32067.2525,0.00000,45.38625,0.0
...,...,...,...,...,...,...,...,...,...,...,...
190175,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-01T23:00:00.000Z,13160.7880,13160.7880,13117.2080,12286.2405,830.96780,43.58000,0.0
190176,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-02T00:00:00.000Z,12677.8850,12677.8850,12634.3050,11803.3875,830.91725,43.58000,0.0
190177,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-02T01:00:00.000Z,11944.9425,11944.9425,11901.1175,11071.8195,829.29760,43.82530,0.0
190178,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-02T02:00:00.000Z,10973.6000,10973.6000,10930.0200,10103.2540,826.76605,43.58000,0.0


Modelo atualizado

In [63]:
import pandas as pd

failure = False; i = 2000
df_carga = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/carga_energia_di/CARGA_ENERGIA_{i}.csv'
    try:
        # Lendo o CSV diretamente da URL com delimitador ';'
        dados_carga = pd.read_csv(url, delimiter=';')
    except Exception as e:
        # print(f"Erro ao carregar o arquivo CSV: {e}")
        failure = True
    if i == 2000:
        df_carga = dados_carga
    elif failure == False: 
        df_carga = pd.concat([df_carga, dados_carga])
    i = i + 1
df_carga.drop(columns= 'nom_subsistema', inplace=True)
df_carga = df_carga.reset_index(drop=True)
df_carga.replace({'SE': 'SE/CO'}, inplace=True)
df_carga

,id_subsistema,din_instante,val_cargaenergiamwmed
0,N,2000-01-01,2243.512500
1,NE,2000-01-01,4646.370833
2,S,2000-01-01,4800.650000
3,SE/CO,2000-01-01,19045.995833
4,N,2000-01-02,2259.808333
...,...,...,...
36344,SE/CO,2024-11-15,NaN
36345,N,2024-11-16,NaN
36346,NE,2024-11-16,NaN
36347,S,2024-11-16,NaN


In [64]:
df_carga.to_csv('Carga_Consumo_atualizado.csv')

# ENA

ENA + MLT

In [ ]:
import pandas as pd

#TEM QUE SER *ESSE* ARQUIVO
historico = pd.read_excel("Enas_Subsistemas_1931-2022.xlsx")

# Função para calcular médias, mínimos e máximos, e atualizar porcentagens
def calcular_estatisticas_e_atualizar_porcentagens(df, novo_ano, novos_valores):

    if novo_ano in df['Ano'].values:
        print(f"O ano {novo_ano} já existe no DataFrame. Nenhuma ação será realizada.")
        return df
    df = df.drop(columns=['Subsistema', '(min)MWmed',	'(min)% med.',	'(max)MWmed', '(max)% med.','(jan)% med.', '(fev)% med.', '(mar)% med.', '(abr)% med.', '(mai)% med.','(jun)% med.', '(jul)% med.', '(ago)% med.', '(set)% med.', '(out)% med.', '(nov)% med.', '(dez)% med.',])
    try : df =  df.drop(columns= ['(jandez)MWmed','(jandez)% med.'])
    except : df =  df.drop(columns= ['(jnd)MWmed','(jnd)% med.'])
    df = df[:-1]
    nova_linha = {'Ano': novo_ano}
    for mes, valor in zip(['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed', '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed'], novos_valores):
        nova_linha[mes] = valor
        
    df = pd.concat([df, pd.DataFrame([nova_linha])], ignore_index=True)
        # Drop the last row (old mean) if it exists
    df['(min)MWmed'] = df.iloc[:, 1:13].min(axis=1)
    df['(jnd)MWmed'] = df.iloc[:, 1:13].mean(axis=1)
    df['(max)MWmed'] = df.iloc[:, 1:13].max(axis=1)
    meses = ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed', '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed']
    mjdm = ['(min)MWmed', '(jnd)MWmed', '(max)MWmed']
    ano_atual = "media"  # Próximo ano

    # Calcular médias mensais
    media_row = {'Ano': ano_atual}
    for mes in meses:
        media_row[mes] = df[mes].mean()  # Média dos valores para cada mês
        
    means = [df[mes].mean() for mes in meses]

    for idx in range(len(df)):
        for i in range(len(meses)):
            value_col = df.iloc[idx][meses[i]]  # Valor atual
            if pd.notna(value_col) and means[i] != 0:  # Previne divisão por zero
                df.loc[idx, f'{meses[i][0:5]}% med.'] = (value_col / means[i]) * 100
            else:
                df.loc[idx, f'{meses[i][0:5]}% med.'] = 100  # Define como None se valor for NaN ou média for 0

    means2 = df['(jnd)MWmed'].mean()

    for idx in range(len(df)):
        for i in range(len(mjdm)):
            value_col = df.iloc[idx][mjdm[i]]  # Valor atual
            if pd.notna(value_col) and means2 != 0:  # Previne divisão por zero
                df.loc[idx, f'{mjdm[i][0:5]}% med.'] = (value_col / means2) * 100
            else:
                df.loc[idx, f'{mjdm[i][0:5]}% med.'] = 100 
    
    # Adiciona a nova linha com as estatísticas
    df = pd.concat([df, pd.DataFrame([media_row])], ignore_index=True)
    df = df.fillna(100)    

    return df

def inicio_SE(historicoSE):

    # Escreva os valores para a região SE na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [71658, 78130, 52904,  41024, 26938, 24249, 16822, 16036, 16085, 24508, 24449, 43079]

    historicoSE = historicoSE[historicoSE['Ano'] != 2022]

    # Calcula estatísticas e atualiza porcentagens
    historicoSE = calcular_estatisticas_e_atualizar_porcentagens(historicoSE, 2022, values_input)
    historicoSE['Subsistema'] = "SE"
    return historicoSE

def inicio_N(historicoN):

    # Escreva os valores para a região N na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [35162, 30798, 32737,  29655, 17111, 8829, 4573, 2739, 1763, 1718, 3841, 10253]

    historicoN = historicoN[historicoN['Ano'] != 2022]


    # Calcula estatísticas e atualiza porcentagens
    historicoN = calcular_estatisticas_e_atualizar_porcentagens(historicoN, 2022, values_input)
    historicoN['Subsistema'] = "N"
    return historicoN

def inicio_S(historicoS):

    # Escreva os valores para a região S na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [2802, 2751, 6990,  10176, 18906, 24482, 7823, 12663, 9727, 20738, 7820, 8163]

    historicoS = historicoS[historicoS['Ano'] != 2022]

    # Calcula estatísticas e atualiza porcentagens
    historicoS = calcular_estatisticas_e_atualizar_porcentagens(historicoS, 2022, values_input)
    historicoS['Subsistema'] = "S"
    return historicoS

def inicio_NE(historicoNE):

    # Escreva os valores para a região NE na ordem dos meses (jan, fev, mar, ..., dez)
    values_input = [17924, 20879, 17281,  7449, 3678, 2857, 2445, 2056, 2234, 1912, 4270, 9539]

    historicoNE = historicoNE[historicoNE['Ano'] != 2022]

    # Calcula estatísticas e atualiza porcentagens
    historicoNE = calcular_estatisticas_e_atualizar_porcentagens(historicoNE, 2022, values_input)
    historicoNE['Subsistema'] = "NE"
    return historicoNE

# Categorização e inicialização do histórico do subsistema SE
historicoSE = historico[historico['Subsistema'] == 'SECO']
historicoSE = inicio_SE(historicoSE)

# Escreva os valores para a região SE na ordem dos meses (jan, fev, mar, ..., dez)
values_inputSE = [71658, 78130, 52904,  41024, 26938, 24249, 16822, 16036, 16085, 24508, 24449, 43079]

# Escreva o ano que você deseja adicionar
anoSE = 2022

# Calcula estatísticas e atualiza porcentagens
historicoSE = calcular_estatisticas_e_atualizar_porcentagens(historicoSE, anoSE, values_inputSE)
historicoSE['Subsistema'] = "SE"

# Categorização do histórico do subsistema N
historicoN = historico[historico['Subsistema'] == 'N']
historicoN = inicio_N(historicoN)

 # Escreva os valores para a região N na ordem dos meses (jan, fev, mar, ..., dez)
values_input = [35162, 30798, 32737,  29655, 17111, 8829, 4573, 2739, 1763, 1718, 3841, 10253]

# Escreva o ano que você deseja adicionar
ano = 2022

# Calcula estatísticas e atualiza porcentagens
historicoN = calcular_estatisticas_e_atualizar_porcentagens(historicoN, ano, values_input)
historicoN['Subsistema'] = "N"


# Categorização do histórico do subsistema S
historicoS = historico[historico['Subsistema'] == 'S']
historicoS = inicio_S(historicoS)

# Escreva os valores para a região S na ordem dos meses (jan, fev, mar, ..., dez)
values_inputS = [2802, 2751, 6990,  10176, 18906, 24482, 7823, 12663, 9727, 20738, 7820, 8163]

# Escreva o ano que você deseja adicionar
anoS = 2022
# Calcula estatísticas e atualiza porcentagens

historicoS = calcular_estatisticas_e_atualizar_porcentagens(historicoS, anoS, values_inputS)
historicoS['Subsistema'] = "S"

# Categorização do histórico do subsistema NE
historicoNE = historico[historico['Subsistema'] == 'NE']
historicoNE = inicio_NE(historicoNE)

 # Escreva os valores para a região NE na ordem dos meses (jan, fev, mar, ..., dez)
values_inputNE = [17924, 20879, 17281,  7449, 3678, 2857, 2445, 2056, 2234, 1912, 4270, 9539]

# Escreva o ano que você deseja adicionar
anoNE = 2022

# Calcula estatísticas e atualiza porcentagens
historicoNE = calcular_estatisticas_e_atualizar_porcentagens(historicoNE, anoNE, values_inputNE)
historicoNE['Subsistema'] = "NE"

df_mlt = pd.concat([historicoSE, historicoS, historicoN, historicoNE]).reset_index(drop=True)
df_mlt.replace({'SE': 'SE/CO'}, inplace=True)


failure = False; i = 2000
df_earm = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/ena_subsistema_di/ENA_DIARIO_SUBSISTEMA_{i}.csv'
    try:
        # Lendo o CSV diretamente da URL com delimitador ';'
        dados_ena = pd.read_csv(url, delimiter=';')
    except Exception as e:
        # print(f"Erro ao carregar o arquivo CSV: {e}")
        failure = True
    if i == 2000:
        df_ena = dados_ena
    elif failure == False: 
        df_ena = pd.concat([df_ena, dados_ena])
    i = i + 1
df_ena.replace({'NORDESTE': 'NE', 'NORTE': 'N','SUL': 'S','SUDESTE': 'SE'}, inplace=True)
df_ena = df_ena.drop(columns=["ena_bruta_regiao_percentualmlt", "ena_armazenavel_regiao_percentualmlt"])
df_ena['ena_data'] = pd.to_datetime(df_ena['ena_data'])
df_ena['month'] = df_ena['ena_data'].dt.month
def month_value_condition(row, df, ena):
    month = row['month']
    sub = row['nom_subsistema']
    ena_value = row[ena]
    
    subsistema_columns = {
        'SE': ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed', 
               '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed'],
        'N': ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed',
              '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed'],
        'NE': ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed',
               '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed'],
        'S': ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed',
              '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed']
    }
    
    if sub in subsistema_columns:
        month_column = subsistema_columns[sub][month - 1]  
        value = ena_value * 100 / df.loc[(df['Subsistema'] == sub) & (df['Ano'] == 'media'), month_column].values[0]
        return value
    else:
        return "Invalid subsistema"

df_ena['ena_bruta_regiao_percentualmlt'] = df_ena.apply(lambda row: month_value_condition(row, df_mlt, 'ena_bruta_regiao_mwmed'), axis=1)
df_ena['ena_armazenavel_regiao_percentualmlt'] = df_ena.apply(lambda row: month_value_condition(row, df_mlt, 'ena_armazenavel_regiao_mwmed'), axis=1)
df_ena = df_ena.drop(columns=['nom_subsistema']).reset_index(drop=True)
df_ena.replace({'SE': 'SE/CO'}, inplace=True)
aux = df_ena.groupby(['id_subsistema', 'month'])['ena_bruta_regiao_mwmed'].agg(['max', 'min']).reset_index()
df_ena = pd.merge(df_ena, aux, on=['id_subsistema', 'month'], how='left')
df_ena


O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.
O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.
O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.
O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.


,id_subsistema,ena_data,ena_bruta_regiao_mwmed,ena_armazenavel_regiao_mwmed,month,ena_bruta_regiao_percentualmlt,ena_armazenavel_regiao_percentualmlt,max,min
0,N,2000-01-01,10392.561523,10381.552734,1,65.642536,65.573001,39296.804688,1878.000000
1,NE,2000-01-01,10174.402344,9990.510742,1,75.481418,74.117171,33141.000000,1859.000000
2,S,2000-01-01,1772.415039,1772.415039,1,23.482726,23.482726,30770.000000,1490.378052
3,SE/CO,2000-01-01,27179.791016,27097.957031,1,41.306259,41.181892,110765.000000,16210.000000
4,N,2000-01-02,10888.611328,10867.503906,1,68.775736,68.642415,39296.804688,1878.000000
...,...,...,...,...,...,...,...,...,...
36355,SE/CO,2024-11-18,43369.285156,42410.125000,11,138.046281,134.993234,55905.203125,13044.000000
36356,N,2024-11-19,3117.352051,3104.507080,11,76.726177,76.410029,9020.367188,739.000000
36357,NE,2024-11-19,4049.818115,4049.818115,11,76.712062,76.712062,11723.000000,618.000000
36358,S,2024-11-19,6031.296875,5876.383789,11,64.773728,63.110024,102106.929688,1509.647949


In [27]:
df_ena.to_csv('Ena_atualizado.csv')
df_mlt.to_csv("Mlt_atualizado.csv")

In [2]:
import pandas as pd
pd.read_csv("Mlt_atualizado.csv").drop(columns='Unnamed: 0')

,Ano,(jan)MWmed,(fev)MWmed,(mar)MWmed,(abr)MWmed,(mai)MWmed,(jun)MWmed,(jul)MWmed,(ago)MWmed,(set)MWmed,...,(jul)% med.,(ago)% med.,(set)% med.,(out)% med.,(nov)% med.,(dez)% med.,(min)% med.,(jnd)% med.,(max)% med.,Subsistema
0,1931,69359.690000,103272.430000,107218.270000,80246.410000,55019.580000,40765.81000,31993.570000,25434.170000,25948.880000,...,125.101976,123.944948,132.210640,117.091602,101.535843,99.071047,60.844401,128.873962,256.490831,SE
1,1932,69378.190000,76951.880000,68566.180000,51823.270000,40237.380000,35224.44000,27384.850000,21334.670000,17898.940000,...,107.080856,103.967401,91.195856,107.101383,100.543753,121.880783,42.818393,104.518598,184.086646,SE
2,1933,79952.560000,68553.650000,58429.420000,53341.080000,38425.430000,28005.79000,21773.810000,17187.930000,16172.280000,...,85.140441,83.759646,82.398450,88.931164,72.262547,92.487523,38.687824,93.700592,191.264964,SE
3,1934,57587.480000,52748.750000,52134.030000,43281.560000,30991.250000,22912.10000,18645.760000,13764.920000,13885.410000,...,72.909070,67.078748,70.746751,64.036409,58.436174,96.887221,32.928863,76.953285,137.762534,SE
4,1935,72107.460000,91347.300000,80328.830000,71662.200000,44950.720000,33590.26000,25174.410000,23909.130000,21174.440000,...,98.437544,116.513174,107.884666,161.025812,97.634865,82.118237,50.654144,114.122940,218.523810,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,2019,5179.260000,3654.660000,6240.260000,6429.000000,4044.060000,2370.23000,1872.030000,1629.040000,1307.960000,...,49.780445,49.679218,44.599310,32.112688,24.577186,47.956864,13.207630,42.733419,82.908776,NE
368,2020,6104.200000,11763.360000,13678.350000,11753.790000,5592.160000,3418.92000,2444.370000,2198.020000,1745.150000,...,64.999934,67.030838,59.506779,57.144129,89.278287,45.942497,22.505561,74.960439,176.396836,NE
369,2021,6103.860000,6022.270000,9855.850000,4107.000000,2683.490000,1596.91000,1613.650000,1359.640000,1265.460000,...,42.909684,41.463594,43.150129,41.132936,78.305507,100.397218,16.319449,53.654954,127.321779,NE
370,2022,17924.000000,20879.000000,17281.000000,7449.000000,3678.000000,2857.00000,2445.000000,2056.000000,2234.000000,...,65.016687,62.699794,76.175769,59.951042,80.882769,97.001603,24.657269,99.432929,269.256858,NE


# EARM

In [ ]:
import pandas as pd

failure = False; i = 2000
df_earm = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/ear_subsistema_di/EAR_DIARIO_SUBSISTEMA_{i}.csv'
    try:
        # Lendo o CSV diretamente da URL com delimitador ';'
        dados_earm = pd.read_csv(url, delimiter=';')
    except Exception as e:
        # print(f"Erro ao carregar o arquivo CSV: {e}")
        failure = True
    if i == 2000:
        df_earm = dados_earm
    elif failure == False: 
        df_earm = pd.concat([df_earm, dados_earm])
    i = i + 1
df_earm.drop(columns= 'nom_subsistema', inplace=True)
sum_ear_verif_subsistema_mwmes = df_earm['ear_verif_subsistema_mwmes'].sum()
sum_ear_max_subsistema = df_earm['ear_max_subsistema'].sum()

# Step 2: Add a new column with the ratio
df_earm['verif_max_ratio'] = (sum_ear_verif_subsistema_mwmes / sum_ear_max_subsistema)*100

# If you want to add the total ratio as a new row (optional):
# df_earm.loc['Total'] = df_earm[['ear_verif_subsistema_mwmes', 'ear_max_subsistema']].sum()
# df_earm['verif_max_ratio'] = sum_ear_verif_subsistema_mwmes / sum_ear_max_subsistema

df_earm.reset_index(drop=True, inplace=True)
df_earm.replace({'SE': 'SE/CO'}, inplace=True)
df_earm

,id_subsistema,ear_data,ear_max_subsistema,ear_verif_subsistema_mwmes,ear_verif_subsistema_percentual,verif_max_ratio
0,NE,2000-01-01,49967.000000,11171.000000,22.360001,54.177739
1,N,2000-01-01,12311.000000,7656.000000,62.189999,54.177739
2,SE/CO,2000-01-01,155759.000000,27853.000000,17.879999,54.177739
3,S,2000-01-01,13976.000000,7094.000000,50.759998,54.177739
4,NE,2000-01-02,49967.000000,11395.000000,22.809999,54.177739
...,...,...,...,...,...,...
36343,S,2024-11-15,20459.242188,13147.633789,64.262604,54.177739
36344,NE,2024-11-16,51691.226562,22825.224609,44.156898,54.177739
36345,N,2024-11-16,15302.396484,8682.454102,56.739201,54.177739
36346,SE/CO,2024-11-16,204615.328125,86904.726562,42.472198,54.177739


In [68]:
df_earm.to_csv('EARM_atualizado.csv')

# PLD

In [2]:
pldH = pd.read_csv('PLD Horário Comercial Historico.csv')
pldH

,Unnamed: 0,Hora,Submercado,Data,Valor
0,0,0,N,2001-06-30,684
1,1,1,N,2001-06-30,684
2,2,2,N,2001-06-30,684
3,3,3,N,2001-06-30,684
4,4,4,N,2001-06-30,684
...,...,...,...,...,...
815731,815731,19,SECO,2024-10-04,643
815732,815732,20,SECO,2024-10-04,"639,75"
815733,815733,21,SECO,2024-10-04,"634,11"
815734,815734,22,SECO,2024-10-04,"624,42"


In [33]:
import pandas as pd

pld = pd.read_csv('modelo_pld.csv', delimiter = ";") #arquivo a ser adicionado

# Primeiro, convertemos o DataFrame de "longo" para "largo"
pld = pld.melt(id_vars=['Hora', 'Submercado'], var_name='Data', value_name='Valor')

# Ajustar o formato da coluna 'Data' para datetime se necessário
pld['Data'] = pd.to_datetime(pld['Data'], format='%d/%m/%Y')

# Limpando o Submercado
submercados = {
    'NORTE': 'N',
    'SUDESTE': 'SE/CO',
    'SUL': 'S',
    'NORDESTE': 'NE'
}
pld['Submercado'] = pld['Submercado'].replace(submercados)

# Formatando a Data
pld['Data'] = pd.to_datetime(pld['Data'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
pld['Valor'] = pld['Valor'].str.replace('.', '', regex=False)
pld['Valor'] = pld['Valor'].str.replace(',', '.', regex=False)
# Selecionando as colunas na ordem desejada
pldH = pd.read_csv('PLD Horário Comercial Historico.csv')
pldH = pldH[['Hora', 'Submercado', 'Data', 'Valor']]
# Visualizar o DataFrame transposto
pld = pd.concat([pldH, pld]).drop_duplicates(subset=['Hora', 'Submercado', 'Data'], keep='first')
pld['Submercado'] = pld['Submercado'].astype(str)
pldn = pld[pld['Submercado'] == 'N'].reset_index(drop=True)
pldne = pld[pld['Submercado'] == 'NE'].reset_index(drop=True)
plds = pld[pld['Submercado'] == 'S'].reset_index(drop=True)
pldseco = pld[pld['Submercado'] == 'SE/CO'].reset_index(drop=True)
pld = pd.concat([pldseco, plds, pldne, pldn]).reset_index(drop=True)
pld['Valor'] = pd.to_numeric(pld['Valor'], errors='coerce')

pld



C:\Users\Fernando\AppData\Local\Temp\ipykernel_35320\2405683548.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  pldH = pd.read_csv('PLD Horário Comercial Historico.csv')


,Hora,Submercado,Data,Valor
0,0,SE/CO,2001-06-30,684.00
1,1,SE/CO,2001-06-30,684.00
2,2,SE/CO,2001-06-30,684.00
3,3,SE/CO,2001-06-30,684.00
4,4,SE/CO,2001-06-30,684.00
...,...,...,...,...
815731,19,N,2024-10-04,644.90
815732,20,N,2024-10-04,642.68
815733,21,N,2024-10-04,634.13
815734,22,N,2024-10-04,624.43


In [5]:
import pandas as pd
pldz = pd.read_csv('PLD Horário Comercial Historico.csv')
pldz = pldz[pldz["Submercado"] == 'NE']
pldz = pldz[pldz["Data"] == '2024-10-02']
pldz

,Unnamed: 0,Hora,Submercado,Data,Valor
611730,611730,0,NE,2024-10-02,611.24
611731,611731,1,NE,2024-10-02,603.89
611732,611732,2,NE,2024-10-02,599.27
611733,611733,3,NE,2024-10-02,599.44
611734,611734,4,NE,2024-10-02,600.10
611735,611735,5,NE,2024-10-02,604.40
611736,611736,6,NE,2024-10-02,599.27
611737,611737,7,NE,2024-10-02,61.07
611738,611738,8,NE,2024-10-02,61.07
611739,611739,9,NE,2024-10-02,173.80


SUBSTITUIÇÃO DE CÉLULA

In [14]:
def update_value(df, hora, submercado, data, new_value):
    condition = (df['Hora'] == hora) & (df['Submercado'] == submercado) & (df['Data'] == data)
    if df[condition].empty:
        print("No matching row found.")
    else:
        df.loc[condition, 'Valor'] = new_value
        print(f"Updated 'Valor' for Hora={hora}, Submercado={submercado}, Data={data} to {new_value}.")

#update_value(pld, 1, 'N', '2001-06-30', 700) 
#formato: (pld, hora, 'submercado', 'YYYY-MM-DD', valor correto)



EXPORTAÇÃO

In [36]:
pld.to_csv('PLD Horário Comercial Historico.csv')

# Tarifas

In [38]:
import pandas as pd
tarifa = pd.read_excel("Histórico Tarifas Distribuidoras (desde 2010).xlsx")
tarifa = tarifa[tarifa['Base Tarifária'] == 'Tarifa de Aplicação']
tarifa = tarifa[(tarifa['Modalidade'] == 'Azul') | (tarifa['Modalidade'] == 'Verde')]
tarifa = tarifa[(tarifa['Detalhe'] == 'Não se aplica') | (tarifa['Detalhe'] == 'APE')]
tarifa = tarifa[tarifa['Classe'] == 'Não se aplica']
tarifa = tarifa[tarifa['Subclasse'] == 'Não se aplica']
tarifa = tarifa[tarifa['Acessante'] == 'Não se aplica']
tarifa = tarifa[tarifa['Subgrupo'].str.contains("A", na=False)]
tarifa['Modalidade'] = tarifa.apply(
    lambda row: f"{row['Modalidade']} {row['Detalhe']}" if row['Detalhe'] == 'APE' else row['Modalidade'] 
    , axis=1)
tarifa = tarifa.drop(columns=['Resolução ANEEL', 'Classe', 'Subclasse', 'Acessante', 'Base Tarifária', 'Detalhe'])
tarifa

,Sigla,Início Vigência,Fim Vigência,Subgrupo,Modalidade,Posto,Unidade,TUSD,TE
1,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul APE,Fora ponta,R$/kW,5.01,0.00
3,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul,Fora ponta,R$/kW,5.01,0.00
5,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul,Fora ponta,R$/MWh,16.67,0.00
7,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul,Fora ponta seca,R$/MWh,16.67,144.52
9,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul,Fora ponta úmida,R$/MWh,16.67,129.11
...,...,...,...,...,...,...,...,...,...
252779,Sulgipe,2024-05-22,2025-05-21,A4,Verde,Fora ponta,R$/MWh,63.81,214.42
252782,Sulgipe,2024-05-22,2025-05-21,A4,Verde APE,Não se aplica,R$/kW,22.80,0.00
252784,Sulgipe,2024-05-22,2025-05-21,A4,Verde,Não se aplica,R$/kW,22.80,0.00
252786,Sulgipe,2024-05-22,2025-05-21,A4,Verde APE,Ponta,R$/MWh,1539.98,0.00


In [53]:
bandeiras = pd.read_excel("2024 09 - HISTÓRICO BANDEIRAS TARIFÁRIAS TODOS SUBMERCADOS (jan2014~).xlsx")
# Calculate the midpoint for rows and columns
mid_row = bandeiras.shape[0] // 2
mid_col = bandeiras.shape[1] // 2

# Split the DataFrame into four parts
df_top_left = bandeiras.iloc[:mid_row, :mid_col]
df_top_right = bandeiras.iloc[:mid_row, mid_col:]
df_bottom_left = bandeiras.iloc[mid_row:, :mid_col]
df_bottom_right = bandeiras.iloc[mid_row:, mid_col:]

# Display the four parts
import pandas as pd

def bottom(df):
    # Exclude the first row and column
    df_processed = df.iloc[1:, 1:]
    
    # Transpose the DataFrame
    df_transposed = df_processed.transpose()
    
    # Set the first row as the header
    df_transposed.columns = df_transposed.iloc[0]
    df_final = df_transposed[1:]  # Exclude the new header row
    
    return df_final.reset_index(drop=True)

def top(df):
    # Excluir a primeira coluna
    df_processed = df.iloc[:13, 1:]
    
    # Transpor o DataFrame
    df_transposed = df_processed.transpose()
    
    # Definir a primeira linha como cabeçalho
    df_transposed.columns = df_transposed.iloc[0]
    df_final = df_transposed[1:]  # Excluir a nova linha de cabeçalho
    
    return df_final.reset_index(drop=True)

bandeiraSeco = top(df_top_left)
bandeiraNordeste = top(df_top_right)
bandeiraSul = bottom(df_bottom_left)
bandeiraNorte = bottom(df_bottom_right)
bandeiraNorte


Unnamed: 14,NORTE,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,2014,Amarela,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
1,2015,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
2,2016,Vermelha,Vermelha,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Amarela,Verde
3,2017,Verde,Verde,Amarela,Vermelha 1,Vermelha 1,Verde,Amarela,Vermelha 1,Amarela,Vermelha 2,Vermelha 2,Vermelha 1
4,2018,Verde,Verde,Verde,Verde,Amarela,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Amarela,Verde
5,2019,Verde,Verde,Verde,Verde,Amarela,Verde,Amarela,Vermelha 1,Vermelha 1,Amarela,Vermelha 1,Amarela
6,2020,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Vermelha 2
7,2021,Amarela,Amarela,Amarela,Amarela,Vermelha 1,Vermelha 2,Vermelha 2,Vermelha 2,Escassez,Escassez,Escassez,Escassez
8,2022,Escassez,Escassez,Escassez,Escassez,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde
9,2023,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde


In [58]:
bandeiraSeco

Unnamed: 1,SE/CO,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,2014,Amarela,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
1,2015,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
2,2016,Vermelha,Vermelha,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Amarela,Verde
3,2017,Verde,Verde,Amarela,Vermelha 1,Vermelha 1,Verde,Amarela,Vermelha 1,Amarela,Vermelha 2,Vermelha 2,Vermelha 1
4,2018,Verde,Verde,Verde,Verde,Amarela,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Amarela,Verde
5,2019,Verde,Verde,Verde,Verde,Amarela,Verde,Amarela,Vermelha 1,Vermelha 1,Amarela,Vermelha 1,Amarela
6,2020,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Vermelha 2
7,2021,Amarela,Amarela,Amarela,Amarela,Vermelha 1,Vermelha 2,Vermelha 2,Vermelha 2,Escassez,Escassez,Escassez,Escassez
8,2022,Escassez,Escassez,Escassez,Escassez,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde
9,2023,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde


In [56]:
def substituir_valor(df, ano, mes, novo_valor):
    # Mapeia os meses para suas respectivas colunas
    meses_map = {
        'JAN': 1,
        'FEV': 2,
        'MAR': 3,
        'ABR': 4,
        'MAI': 5,
        'JUN': 6,
        'JUL': 7,
        'AGO': 8,
        'SET': 9,
        'OUT': 10,
        'NOV': 11,
        'DEZ': 12
    }

    # Verifica se o ano já está no DataFrame
    if ano in df.iloc[:, 0].values:
        # Encontra o índice do ano
        idx = df[df.iloc[:, 0] == ano].index[0]

        # Verifica se o mês está mapeado
        if mes in meses_map:
            # Atualiza o valor na célula específica
            coluna = meses_map[mes]
            df.iat[idx, coluna] = novo_valor
        else:
            print("Mês inválido")
            return df
    else:
        # Se o ano não está presente, adiciona uma nova linha
        nova_linha = [ano] + [0] * (df.shape[1] - 1)  # Cria nova linha com zeros
        df.loc[len(df)] = nova_linha  # Adiciona a nova linha
        idx = len(df) - 1  # Atualiza o índice da nova linha

        # Verifica se o mês está mapeado
        if mes in meses_map:
            coluna = meses_map[mes]
            df.iat[idx, coluna] = novo_valor
        else:
            print("Mês inválido")
            return df
    
    return df

bandeiraSeco = substituir_valor(bandeiraSeco, 2024, 'DEZ', 'oi') #dataframe a ser alterada, yyyy, 'MMM', 'valor adicionado'
# bandeiraSeco = substituir_valor(bandeiraSeco, 2025, 'MAI', 'TESTE2')
bandeiraSeco

Unnamed: 1,SE/CO,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,2014,Amarela,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
1,2015,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
2,2016,Vermelha,Vermelha,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Amarela,Verde
3,2017,Verde,Verde,Amarela,Vermelha 1,Vermelha 1,Verde,Amarela,Vermelha 1,Amarela,Vermelha 2,Vermelha 2,Vermelha 1
4,2018,Verde,Verde,Verde,Verde,Amarela,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Amarela,Verde
5,2019,Verde,Verde,Verde,Verde,Amarela,Verde,Amarela,Vermelha 1,Vermelha 1,Amarela,Vermelha 1,Amarela
6,2020,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Vermelha 2
7,2021,Amarela,Amarela,Amarela,Amarela,Vermelha 1,Vermelha 2,Vermelha 2,Vermelha 2,Escassez,Escassez,Escassez,Escassez
8,2022,Escassez,Escassez,Escassez,Escassez,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde
9,2023,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde


In [44]:
bandeiraNorte.to_csv('bandeira_N.csv')
bandeiraSul.to_csv('bandeira_S.csv')
bandeiraNordeste.to_csv('bandeira_NE.csv')
bandeiraSeco.to_csv('bandeira_SECO.csv')

In [39]:
tarifa.to_csv("tarifa_atualizado.csv")

# Curva Forward

In [60]:
import pandas as pd

import pandas as pd

df = pd.read_excel("DADOS CURVA FORWARD CONSOLIDADOS.xlsx")

def soma_base(df, dia, subregiao, a_plus):
    # Converter o dia para o formato datetime
    dia = pd.to_datetime(dia)

    # Filtrar o DataFrame pelo dia
    df_filtrado = df[(df['DIA'] == dia)]

    if df_filtrado.empty:
        return f"Não há dados para a data {dia.date()}."

    # Definir a coluna com base na subregião e A+
    if subregiao.lower() == 'norte':
        coluna_base = f'Norte Convencional {a_plus}'
    elif subregiao.lower() == 'sul':
        coluna_base = f'Sul Convencional {a_plus}'
    elif subregiao.lower() == 'nordeste':
        coluna_base = f'Nordeste Convencional {a_plus}'
    elif subregiao.lower() == 'se/co':
        coluna_base = f'SE/CO Convencional {a_plus}'
    else:
        return "Subregião inválida."

    # Verificar se a coluna existe
    if coluna_base not in df.columns:
        return f"A coluna {coluna_base} não existe no DataFrame."
    
    # Pegar o valor da coluna SE/CO Convencional {a_plus}
    valor_base = df_filtrado[f'SE/CO Convencional {a_plus}'].iloc[0]

    # Calcular a soma
    soma = df_filtrado[coluna_base].sum()

    # Arredondar o resultado para 2 casas decimais
    resultado = round(soma + valor_base, 2)
    
    return resultado


# Selecione apenas as colunas necessárias e renomeie-as
df_organizado = df[['DIA', 'SEMANA', 'MÊS', 
                    'SE/CO CONVENCIONAL', 
                    'Unnamed: 4',  
                    'Unnamed: 5',  
                    'Unnamed: 6', 
                    'SUL CONVENCIONAL', 
                    'Unnamed: 8',  
                    'Unnamed: 9',  
                    'Unnamed: 10',
                    'NORDESTE CONVENCIONAL', 
                    'Unnamed: 12',  
                    'Unnamed: 13',  
                    'Unnamed: 14', 
                    'NORTE CONVENCIONAL', 
                    'Unnamed: 16',  
                    'Unnamed: 17',  
                    'Unnamed: 18',
                    'SE/CO INCENTIVADA',
                    'Unnamed: 20',  
                    'Unnamed: 21',  
                    'Unnamed: 22']]

# Renomear as colunas
df_organizado.columns = ['DIA', 'SEMANA', 'MÊS', 
                         'SE/CO Convencional A+1', 
                         'SE/CO Convencional A+2', 
                         'SE/CO Convencional A+3', 
                         'SE/CO Convencional A+4', 
                         'Sul Convencional A+1', 
                         'Sul Convencional A+2', 
                         'Sul Convencional A+3', 
                         'Sul Convencional A+4',
                         'Nordeste Convencional A+1', 
                         'Nordeste Convencional A+2', 
                         'Nordeste Convencional A+3', 
                         'Nordeste Convencional A+4',
                         'Norte Convencional A+1', 
                         'Norte Convencional A+2', 
                         'Norte Convencional A+3', 
                         'Norte Convencional A+4',
                         'SE/CO Incentivada A+1',
                         'SE/CO Incentivada A+2',
                         'SE/CO Incentivada A+3',
                         'SE/CO Incentivada A+4']

def aplicar_soma_base(row):
    # Para cada coluna, aplicar soma_base, mas ignorar as colunas específicas
    for col in row.index[3:]:  # Pular as primeiras três colunas (DIA, SEMANA, MÊS)
        # Verifica se a coluna não é uma das que não deve ser alterada
        if col not in ['SE/CO Convencional A+1', 'SE/CO Convencional A+2', 'SE/CO Convencional A+3', 'SE/CO Convencional A+4', 
                       'SE/CO Incentivada A+1', 'SE/CO Incentivada A+2', 'SE/CO Incentivada A+3', 'SE/CO Incentivada A+4']:
            # Extrair subregião e A+
            subregiao, a_plus = col.split(" ")[0].lower(), col.split(" ")[-1]
            valor = soma_base(df_organizado, row['DIA'], subregiao, a_plus)
            row[col] = valor  # Substituir o valor na célula
    return row

def aplicar_soma_incentivada(row):
    """
    Soma os valores da coluna Incentivada nas colunas de Convencional.
    """
    # Somar as colunas Incentivada com as respectivas colunas Convencionais
    for subregiao in ['sul', 'norte', 'nordeste']:
        for a_plus in ['A+1', 'A+2', 'A+3', 'A+4']:
            col_conv = f"{subregiao.capitalize()} Convencional {a_plus}"
            col_incent = f"SE/CO Incentivada {a_plus}"
            
            if col_conv in row.index and col_incent in row.index:
                # Somar os valores da coluna incentivada com a convencional
                row[col_conv] = round(row[col_conv] + row[col_incent], 2)

    return row


# Exibir o DataFrame organizado
df_organizado = df_organizado.drop(index=0).reset_index(drop=True)

# Aplicar a soma base para cada linha
convencional = df_organizado.apply(aplicar_soma_base, axis=1)

# Exibir o resultado
convencional
incentivada = convencional.apply(aplicar_soma_incentivada, axis=1)
convencional = convencional.iloc[:, :-4]
incentivada = incentivada.drop(columns=['SE/CO Convencional A+1', 'SE/CO Convencional A+2', 'SE/CO Convencional A+3', 'SE/CO Convencional A+4'])
incentivada.columns = incentivada.columns.str.replace('Convencional', 'Incentivada')
colunas_seco_incentivada = ['SE/CO Incentivada A+1', 'SE/CO Incentivada A+2', 
                                'SE/CO Incentivada A+3', 'SE/CO Incentivada A+4']
    
# Selecionar as colunas que não são SE/CO Incentivada
colunas_outros = [col for col in incentivada.columns if col not in colunas_seco_incentivada and col not in ['DIA', 'SEMANA', 'MÊS']]

# Reorganizar as colunas colocando SE/CO Incentivada primeiro
colunas_final = ['DIA', 'SEMANA', 'MÊS'] + colunas_seco_incentivada + colunas_outros
incentivada = incentivada[colunas_final]

In [61]:
convencional.to_csv("Curva_convencional.csv")
incentivada.to_csv("Curva_incentivada.csv")

In [62]:
incentivada

,DIA,SEMANA,MÊS,SE/CO Incentivada A+1,SE/CO Incentivada A+2,SE/CO Incentivada A+3,SE/CO Incentivada A+4,Sul Incentivada A+1,Sul Incentivada A+2,Sul Incentivada A+3,Sul Incentivada A+4,Nordeste Incentivada A+1,Nordeste Incentivada A+2,Nordeste Incentivada A+3,Nordeste Incentivada A+4,Norte Incentivada A+1,Norte Incentivada A+2,Norte Incentivada A+3,Norte Incentivada A+4
0,2012-01-02,s01,01/2012,113.50,123.00,124.50,124.50,197.84,213.45,216.45,216.45,194.89,210.28,213.57,213.78,187.73,205.10,208.31,208.50
1,2012-01-09,s02,01/2012,116.50,123.50,126.50,126.50,198.38,212.08,218.08,218.09,195.47,208.95,215.31,215.46,187.92,203.36,209.71,209.85
2,2012-01-16,s03,01/2012,123.50,129.00,130.50,130.50,203.88,217.01,221.01,221.01,200.85,213.99,218.35,218.46,193.09,207.79,212.07,212.21
3,2012-01-23,s04,01/2012,117.00,125.00,126.50,126.00,196.84,213.05,216.05,215.04,193.83,210.00,213.35,212.37,187.06,204.57,207.86,206.94
4,2012-01-30,s05,01/2012,112.00,119.00,120.00,120.00,191.90,206.58,208.58,208.58,189.15,203.96,206.20,206.23,181.91,198.08,200.27,200.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,2024-09-09,s37,09/2024,200.12,189.08,179.87,176.30,366.83,344.63,326.27,319.65,357.89,337.14,319.43,313.29,359.40,337.87,320.08,313.59
663,2024-09-16,s38,09/2024,198.78,186.88,178.24,175.23,364.12,340.58,323.39,317.85,354.63,332.43,316.31,311.24,356.51,333.74,317.23,312.15
664,2024-09-23,s39,09/2024,203.09,189.83,180.13,176.11,373.03,346.55,327.08,319.49,362.55,338.39,319.69,313.18,365.03,339.64,320.72,313.92
665,2024-09-30,s40,09/2024,203.96,188.24,178.05,175.13,374.86,343.83,323.22,317.50,363.90,334.74,315.38,310.33,367.19,336.49,316.56,311.50
